# MLP Model with Keras for trump selection
Model to determine trump selection.
Generate model is used in determination_monte_carlo_tree_search

### Load and initialize Data

In [4]:
import pandas as pd
from pathlib import Path

path_to_data = Path('../ML_sklearn/')
# Import only a fraction of data for efficient testing
data = pd.read_csv(path_to_data / '2018_10_18_trump.csv', header=None)
cards = [
    # Diamonds
    'DA', 'DK', 'DQ', 'DJ', 'D10', 'D9', 'D8', 'D7', 'D6',
    # Hearts
    'HA', 'HK', 'HQ', 'HJ', 'H10', 'H9', 'H8', 'H7', 'H6',
    # Spades
    'SA', 'SK', 'SQ', 'SJ', 'S10', 'S9', 'S8', 'S7', 'S6',
    # Clubs
    'CA', 'CK', 'CQ', 'CJ', 'C10', 'C9', 'C8', 'C7', 'C6'
]

# Forehand (yes = 1, no = 0)
forehand = ['FH']

user = ['user']
trump = ['trump']

data.columns = cards + forehand + user + trump
feature_columns = cards + forehand
data.drop('user', axis='columns', inplace=True)
print(data)

        DA  DK  DQ  DJ  D10  D9  D8  D7  D6  HA  ...  CK  CQ  CJ  C10  C9  C8  \
0        0   0   0   1    1   0   1   1   0   0  ...   0   1   0    0   0   1   
1        0   0   0   0    0   0   0   0   1   1  ...   0   0   1    0   0   0   
2        1   0   0   1    0   0   0   0   0   0  ...   0   1   0    0   0   0   
3        0   0   0   0    0   0   0   0   0   1  ...   0   0   0    1   1   0   
4        0   1   0   0    0   0   0   0   1   1  ...   0   0   1    0   0   0   
...     ..  ..  ..  ..  ...  ..  ..  ..  ..  ..  ...  ..  ..  ..  ...  ..  ..   
359820   0   0   0   0    0   1   0   0   0   0  ...   0   1   0    0   1   0   
359821   1   0   0   0    0   0   0   0   1   1  ...   0   0   0    1   0   0   
359822   1   1   1   0    0   0   0   0   0   0  ...   0   0   0    0   0   1   
359823   0   0   0   0    1   0   1   1   0   0  ...   0   1   1    0   0   1   
359824   0   0   1   0    0   0   0   0   0   1  ...   0   0   0    0   1   0   

        C7  C6  FH  trump  

### Arrange Dataset

In [5]:
dummy1 = pd.get_dummies(data['trump'])
print(dummy1)
trumps = ['D', 'H', 'S', 'C', 'O', 'U', 'P']
dummy1.columns = trumps
dummy1.head()
data = pd.concat([data, dummy1], axis=1).drop('trump', axis=1)
print(data.head())

        0  1  2  3  4  5  6
0       0  0  0  0  0  0  1
1       0  0  0  0  0  1  0
2       0  0  0  0  0  0  1
3       0  0  0  0  0  1  0
4       0  0  0  0  1  0  0
...    .. .. .. .. .. .. ..
359820  0  0  0  0  0  0  1
359821  0  0  1  0  0  0  0
359822  0  0  0  0  1  0  0
359823  0  0  0  1  0  0  0
359824  0  0  0  0  0  0  1

[359825 rows x 7 columns]
   DA  DK  DQ  DJ  D10  D9  D8  D7  D6  HA  ...  C7  C6  FH  D  H  S  C  O  U  \
0   0   0   0   1    1   0   1   1   0   0  ...   0   0   0  0  0  0  0  0  0   
1   0   0   0   0    0   0   0   0   1   1  ...   1   0   0  0  0  0  0  0  1   
2   1   0   0   1    0   0   0   0   0   0  ...   1   1   0  0  0  0  0  0  0   
3   0   0   0   0    0   0   0   0   0   1  ...   0   0   0  0  0  0  0  0  1   
4   0   1   0   0    0   0   0   0   1   1  ...   0   0   1  0  0  0  0  1  0   

   P  
0  1  
1  0  
2  1  
3  0  
4  0  

[5 rows x 44 columns]


### Feature Engineering

In [6]:
for color in 'DHSC':
    # Jack and nine combination
    new_col = '{}_J9'.format(color)
    data[new_col] = data['{}J'.format(color)] & data['{}9'.format(color)]
    feature_columns.append(new_col)

    new_col = '{}_AKQ'.format(color)
    data[new_col] = data['{}A'.format(color)] & data['{}K'.format(color)] & data['{}Q'.format(color)]
    feature_columns.append(new_col)

    new_col = '{}_678'.format(color)
    data[new_col] = data['{}6'.format(color)] & data['{}7'.format(color)] & data['{}8'.format(color)]
    feature_columns.append(new_col)

print(data.head())

   DA  DK  DQ  DJ  D10  D9  D8  D7  D6  HA  ...  D_678  H_J9  H_AKQ  H_678  \
0   0   0   0   1    1   0   1   1   0   0  ...      0     0      0      0   
1   0   0   0   0    0   0   0   0   1   1  ...      0     0      0      0   
2   1   0   0   1    0   0   0   0   0   0  ...      0     0      0      0   
3   0   0   0   0    0   0   0   0   0   1  ...      0     0      0      0   
4   0   1   0   0    0   0   0   0   1   1  ...      0     0      1      0   

   S_J9  S_AKQ  S_678  C_J9  C_AKQ  C_678  
0     0      0      0     0      0      0  
1     0      0      0     0      0      0  
2     0      0      0     0      0      0  
3     0      0      0     0      0      0  
4     0      0      0     0      0      0  

[5 rows x 56 columns]


### Split data into train & test

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data[feature_columns], data[trumps], test_size=0.2,
                                                    stratify=data[trumps], random_state=42, shuffle=True)
print(X_train.shape)
print(X_train.head())
print(y_train.head())

(287860, 49)
        DA  DK  DQ  DJ  D10  D9  D8  D7  D6  HA  ...  D_678  H_J9  H_AKQ  \
273767   0   0   0   0    0   0   1   0   0   0  ...      0     0      0   
174327   0   1   0   0    0   1   0   1   0   0  ...      0     0      0   
15004    0   1   0   0    0   0   1   0   0   1  ...      0     0      0   
174714   0   0   0   1    0   0   0   0   0   1  ...      0     0      0   
61048    0   0   1   0    1   0   0   0   0   0  ...      0     0      0   

        H_678  S_J9  S_AKQ  S_678  C_J9  C_AKQ  C_678  
273767      0     0      0      0     0      0      0  
174327      0     0      0      0     0      0      0  
15004       0     0      0      0     0      0      0  
174714      0     0      0      0     0      0      0  
61048       0     0      0      0     0      0      0  

[5 rows x 49 columns]
        D  H  S  C  O  U  P
273767  0  0  0  0  0  1  0
174327  0  0  0  0  0  0  1
15004   0  1  0  0  0  0  0
174714  0  0  0  0  0  0  1
61048   0  0  0  0  0  0  1


### Build Keras Model

In [9]:
from sklearn.metrics import accuracy_score
from tensorflow import keras
import matplotlib.pyplot as plt

model = keras.Sequential()
dataset_dim = X_train.shape[1]

model.add(keras.layers.Dense(100, activation=keras.activations.relu, input_shape=(dataset_dim,)))
model.add(keras.layers.Dense(300, activation=keras.activations.relu))
model.add(keras.layers.Dense(200, activation=keras.activations.relu))
# model.add(keras.layers.Dense(200, activation=keras.activations.sigmo id))
# model.add(keras.layers.Dense(100, activation=keras.activations.sigmoid))
model.add(keras.layers.Dense(7, activation=keras.activations.softmax))
model.compile(loss=keras.losses.CategoricalCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=100, batch_size=32)


def plot_costs(costs):
    fig, ax = plt.subplots()
    ax.plot(costs)
    ax.set_title("Loss curve")
    plt.show()


plot_costs(history.history["loss"])
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
# f1 = f1_score(y_test, y_pred)

print("Accuracy: %.4f" % accuracy)

Epoch 1/10
8996/8996 [==============================] - 15s 2ms/step - loss: 0.8444 - accuracy: 0.6435
Epoch 2/10
8996/8996 [==============================] - 15s 2ms/step - loss: 0.7999 - accuracy: 0.6580
Epoch 3/10
8996/8996 [==============================] - 16s 2ms/step - loss: 0.7901 - accuracy: 0.6608
Epoch 4/10
8996/8996 [==============================] - 17s 2ms/step - loss: 0.7848 - accuracy: 0.6633
Epoch 5/10
8996/8996 [==============================] - 16s 2ms/step - loss: 0.7812 - accuracy: 0.6646
Epoch 6/10
8996/8996 [==============================] - 16s 2ms/step - loss: 0.7773 - accuracy: 0.6649
Epoch 7/10
8996/8996 [==============================] - 16s 2ms/step - loss: 0.7744 - accuracy: 0.6653
Epoch 8/10
3066/8996 [=========>....................] - ETA: 10s - loss: 0.7680 - accuracy: 0.6679

KeyboardInterrupt: 

### Test Model (Only do onces!!!)

In [39]:
y_pred = model.predict(X_test)
result = model.evaluate(X_test, y_test, batch_size=128)
print("test loss, test acc:", result)
print(y_pred)

563/563 [==============================] - 1s 1ms/step - loss: 0.9145 - accuracy: 0.6203
test loss, test acc: [0.9144771099090576, 0.6202598214149475]
[[5.83510482e-05 5.29910940e-05 5.55350631e-03 ... 1.42302020e-02
  9.43268389e-02 8.85754704e-01]
 [1.31709948e-01 2.20957900e-05 4.56672395e-03 ... 7.83741532e-04
  2.28521172e-02 8.21404636e-01]
 [1.02694735e-01 2.39107336e-04 5.13394771e-04 ... 3.36712750e-04
  1.61454663e-01 6.16042614e-01]
 ...
 [9.19246078e-02 1.09012112e-01 1.87512895e-04 ... 2.06869133e-02
  1.59970648e-03 7.76327133e-01]
 [3.56921606e-04 9.73294228e-02 5.25333635e-05 ... 5.74547127e-02
  1.02158189e-01 3.38580549e-01]
 [3.31786007e-01 1.47132250e-03 6.45667824e-05 ... 2.58079381e-04
  2.00823084e-01 4.65238124e-01]]


### Save model for later use

In [40]:
model.save("mlp_model_v2")

INFO:tensorflow:Assets written to: mlp_model_v2\assets
